In [ ]:
# LIBRARIES IMPORT
import mercury as mr
import ee
import geemap.foliumap as geemap
import os
import datetime
import webbrowser
import logging
import geocoder
import folium
import json
import matplotlib.pyplot as plt
from folium.plugins import Draw
from ipyleaflet import *
from colorama import Fore
from PIL import Image

# PROGRAM MODULES IMPORT
sys.path.insert(0, 'gee//modules//') # gee modules path
sys.path.insert(0, 'osm//') # osm modules path
import create_ee_creds as cec
import txtreading as txtr
import landsat_init as l_init
import sentinel_init as s_init
import modis_init as m_init
import othersds_init as o_init
import customds_init as cd_init
import osm_processing as op

In [ ]:
# PROGRAM PRE-PROCESSING

today = datetime.datetime.today()
time = today.strftime("%Y%m%d-%H.%M.%S")
timestr = str(time)

logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)
handler = logging.FileHandler('gee//gaia-gee-errors.log')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

mr.Markdown(text = '**🌎 GAIA - Geospatial & Aerial Images Analyser [BETA] 🌎**')
mr.Markdown(text = '**🌎 This program solution is not final and some details are potentially to be changed 🌎**')
mr.Markdown(text = '**🌏 Current version - 0.41b 🌏**')

text = 'OSINT-TECHNOLOGIES'
text_params = {
    'fontsize': 10,
    'fontcolor': 'black',
    'bold': True,
    'padding': '3px',
    'background': True,
    'bg_color': 'white',
    'border_radius': '1px',
    'position': 'bottomleft',
}

sources_list = ["Google EE", "OpenStreetMap"]
control_panel_choice = mr.Select(value="Google EE", choices=sources_list, label="Choose the source control panel 🔍")

app  = mr.App(title="GAIA v0.41b [BETA] 🌐",
              description="Open-Source Geospatial & Aerial Images Analyser",
              show_code=False,
              show_prompt=False,
              continuous_update=True,
              static_notebook=False,
              show_sidebar=True,
              full_screen=True,
              allow_download=False)

In [ ]:
# GOOGLE EE PROCESSING

if control_panel_choice.value == "Google EE":
    if str((cec.service_acc_filename).rstrip()) not in os.listdir():
        print(Fore.RED + '[File/directory missing error]. Your EE credentials .JSON file was not found in GAIA directory')
        mr.Stop()
    else:
        key_file = (cec.service_acc_filename).rstrip()
        service_account = (cec.service_acc_mail).rstrip()
        credentials = ee.ServiceAccountCredentials(service_account, key_file)
        ee.Initialize(credentials)
        
    m = geemap.Map()
    m.add_basemap('SATELLITE')
    m.add_basemap('TERRAIN')
    m.add_basemap('HYBRID')

    ee_user_guide = mr.Checkbox(value=False, label="How to start Google EE")
    ds_info = mr.Checkbox(value=False, label='Show datasets knowledge base')

    date_one = mr.Text(value="2022-01-01", label="Enter start date (format YYYY-MM-DD)", rows=1)
    date_two = mr.Text(value="2022-01-02", label="Enter final date (format YYYY-MM-DD)", rows=1)

    max_cloud_covering = mr.Slider(value=80, label='Max cloud cover (%, may affect datasets quality)', min=0, max=100)
    datasets_landsat = mr.Checkbox(value=False, label="Show Landsat datasets")
    datasets_sentinel = mr.Checkbox(value=False, label="Show Sentinel datasets")
    datasets_modis = mr.Checkbox(value=False, label="Show MODIS datasets")
    datasets_other = mr.Checkbox(value=False, label='Show other datasets')
                                 
    try:
        if datasets_landsat.value:
            l_init.landsat_initialization(m, date_one, date_two, max_cloud_covering)
    
        if datasets_sentinel.value:
            s_init.sentinel_initialization(m, date_one, date_two, max_cloud_covering)
        
        if datasets_modis.value:
            m_init.modis_initialization(m, date_one, date_two)
        
        if datasets_other.value:
            o_init.others_initialization(m, date_one, date_two)     
    except ee.EEException as error:
        print(Fore.RED + '[Map plotting/initialization error] - {}'.format(str(error)))
        logger.error(error)
        pass

    manual_input = mr.Checkbox(value=False, label='Open custom GEE dataset menu')
    if manual_input.value:
        try:
            custom_ds = mr.Text(value="-", label="[Custom DS] Enter GEE dataset name", rows=1)
            date_one_custom = mr.Text(value="1972-01-01", label="[Custom DS] Enter start date (format YYYY-MM-DD)", rows=1)
            date_two_custom = mr.Text(value="1973-01-01", label="[Custom DS] Enter final date (format YYYY-MM-DD)", rows=1)
            cd_init.customds_initialization(m, date_one_custom, date_two_custom, custom_ds)
        except ee.EEException as error:
            print(Fore.RED + '[Map plotting/initialization error] - {}'.format(str(error)))
            logger.error(error)
            pass

    save_map_html = mr.Button(label="Save current map [HTML]")
    ee_reg = mr.Button(label="EE registration")
    ee_service_acc = mr.Button(label="Create EE service account")

    if ee_reg.clicked:
        webbrowser.open_new_tab('https://code.earthengine.google.com/register')
    
    if ee_service_acc.clicked:
        webbrowser.open_new_tab('https://console.cloud.google.com/iam-admin/serviceaccounts/')

    m.add_text(text, **text_params)
        
    mr.Markdown(text = "#GOOGLE EARTH ENGINE MAP 🌐")
    mr.Markdown(text = "**DATASETS TIME PERIOD: from {} to {}**".format(date_one.value, date_two.value))
    
    if manual_input.value:
        mr.Markdown(text = "**CUSTOM DATASETS TIME PERIOD: from {} to {}**".format(date_one_custom.value, date_two_custom.value))
    
    if ds_info.value:
        mr.Markdown(text="**LANDSAT BANDS COMBINATIONS ⤵**")
        clb = Image.open('gee//data//clb.png')
        plt.figure(figsize=(7, 7))
        plt.imshow(clb)
        plt.axis('off')
        plt.show()
        mr.Markdown(text="**SENTINEL 2 BANDS COMBINATIONS ⤵**")
        csb = Image.open('gee//data//csb.png')
        plt.figure(figsize=(7, 7))
        plt.imshow(csb)
        plt.axis('off')
        plt.show()
        txtr.ds_info_read()

    if ee_user_guide.value:
        txtr.ee_guide_read()
        
    download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
    if save_map_html.clicked:
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)
        html_file = os.path.join(download_dir, 'GAIA-{}.html'.format(timestr))
        m.to_html(filename=html_file, title='My Map', width='100%', height='880px')

    display(m)

In [ ]:
# OSM PROCESSING

if control_panel_choice.value == "OpenStreetMap":
    mr.Markdown(text = "#OPENSTREETMAP 🌐")
    geojson_file = mr.File(label="Upload GeoJSON file for processing", max_file_size="100MB")
    place_long = mr.Text(value="", label="Place longitude: ", rows=1)
    place_lat = mr.Text(value="", label="Place latitude:  ", rows=1)
    basic_zoom = mr.Slider(value=15, label='Basic map zoom (less = more info on the map): ', min=1, max=20)
    add_datasets = mr.Checkbox(value=False, label="View additional maps (for interactive map only)")
    dual_maps = mr.Checkbox(value=False, label="Enter dual maps mode (for interactive map only)")
    osm_analysis = mr.Checkbox(value=False, label="Open OSM analysis menu")
    lat = place_lat.value
    long = place_long.value
    
    try:
        map_folium = folium.Map(location=[lat, long], zoom_start=basic_zoom.value)
        if geojson_file.filepath is not None:
            try:
                geojson = open(geojson_file.filepath)
                geojson_info = json.load(geojson)
                coordinates = geojson_info['features'][0]['geometry']['coordinates']
                folium.GeoJson(geojson_info, name='geojson').add_to(map_folium)
            except IndexError as error:
                print(Fore.RED + '[GeoJSON file error] - {}'.format(str(error)))
        else:
            pass
            
        if add_datasets.value:
            op.additional_datasets(map_folium)

        op.map_layers(map_folium)
        
        if dual_maps.value:
            map_folium = folium.plugins.DualMap(location=[place_lat.value, place_long.value], zoom_start=basic_zoom.value)
            op.dual_maps_processing(map_folium)
        else:
            folium.LayerControl().add_to(map_folium)

        draw = Draw(export=True)
        draw.add_to(map_folium)
        display(map_folium)
            
        if osm_analysis.value:
            sn_analysis = mr.Checkbox(value=False, label="Visualising street network")
            buildings_analysis = mr.Checkbox(value=False, label="Buildings visualisation")
            map_size = mr.Slider(value=15, label='Map size (NxN):', min=1, max=20)
            place_delta = mr.Text(value="0.1", label="DELTA (default is 0.1):  ", rows=1)
            g = geocoder.osm([place_lat.value, place_long.value], method='reverse')
            mr.Markdown(text = "**PLACE NAME: {}**".format(str(g)))
            #g.json for more information
            op.map_plotting(map_size, place_long.value, place_lat.value, place_delta.value)

            if sn_analysis.value:
                op.sn_analysis(place_long.value, place_lat.value)
            
            if buildings_analysis.value:
                op.building_analysis(place_long.value, place_lat.value)

    except (ValueError, TypeError):
            print('Coordinates was not entered (or entered in incorrect type) or they are within geographical boundaries')